# Notebook Contents

In this notebook the visibility lattice is calculated for the voxelated grid. 
The points of interest along the IJ are made using grashopper in the 3D model which are imported into the notebook
The centroids for the voxels are found using the library https://topogenesis.readthedocs.io/
and intersection percentages are calulated with the context of the building massing and the site using the library https://trimsh.org/trimesh.html

# Initilization

In [1]:
import os
import sys
import topogenesis as tg
import pyvista as pv
import pandas as pd
import trimesh as tm
from itertools import cycle
import numpy as np
import pickle
import numpy.ma as ma
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
from ladybug.sunpath import Sunpath

In [2]:
#Design Details
Self_development_plots_path = os.path.relpath('Site_self_development_plots.obj')
Self_development_backyards_path = os.path.relpath('Site_self_development_backyards.obj')
Site_buildings_path = os.path.relpath('Site_buildings.obj')
Site_green_areas_path = os.path.relpath('Site_green_areas.obj')
Site_roads_path = os.path.relpath('Site_base_roads.obj')
Site_context_shading_path= os.path.relpath('Site_surrounding_buildings_for_shading.obj') 
Context_mesh_for_building_one_path = os.path.relpath('Context_for_building_one.obj') 
Context_mesh_for_building_two_and_three_path = os.path.relpath('Context_for_building_two_three.obj') 
Context_mesh_for_building_four_path = os.path.relpath('Context_for_building_four.obj') 

# Site details
Site_base_path = os.path.relpath('Site_base_block.obj')
Site_surrounding_buildings_path = os.path.relpath('Site_surrounding_buildings.obj')
Site_water_bodies_path = os.path.relpath('Site_water_bodies.obj')
Site_roads_path = os.path.relpath('Site_roads.obj')
Site_other_buildings_path = os.path.relpath('Site_other_buildings.obj')

# load the mesh from file
# Design elements
Self_development_plots_mesh = tm.load(Self_development_plots_path)
Self_development_backyards_mesh = tm.load(Self_development_backyards_path)
Site_building_mesh = tm.load(Site_buildings_path)
Site_green_areas_mesh = tm.load(Site_green_areas_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_context_shading_mesh = tm.load(Site_context_shading_path)
Context_mesh_for_building_one_mesh = tm.load(Context_mesh_for_building_one_path)
Context_mesh_for_building_two_and_three_mesh =tm.load(Context_mesh_for_building_two_and_three_path)
Context_mesh_for_building_four_mesh =tm.load(Context_mesh_for_building_four_path)

#Site elements
Site_base_mesh = tm.load(Site_base_path)
Site_surrounding_buildings_mesh = tm.load(Site_surrounding_buildings_path)
Site_water_bodies_mesh = tm.load(Site_water_bodies_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_other_buildings_mesh = tm.load(Site_other_buildings_path)


# Check if the mesh is watertight
#print(envelope_mesh.is_watertight)
#print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


# Import points on the IJ and Road

In [3]:
# Csv to points 
Complete_file_for_IJ = pd.read_excel('Visibility_to_IJ_points.xlsx', sheet_name=0,engine='openpyxl',header = None )
array_excel_points_for_IJ = Complete_file_for_IJ.to_numpy()

In [4]:
Visibility_sources_for_IJ = array_excel_points_for_IJ.T

In [5]:
# Csv to points 
Complete_file_for_road = pd.read_excel('Visibility_from_roads.xlsx', sheet_name=0,engine='openpyxl',header = None )
array_excel_points_for_roads = Complete_file_for_road.to_numpy()

In [6]:
Visibility_sources_for_road = array_excel_points_for_roads.T

# Viz the Points

In [7]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#f4acb7')
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')


# plotting
p.show(use_ipyvtk=True)
p.add_points( Visibility_sources_for_IJ , color='#ffa500')
p.add_points( Visibility_sources_for_road , color='#2a9d8f')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)00000261C9ACB820

# Import lattice

In [8]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

# Find Voxel Eucledian Coordinates

In [9]:
# convert to lattice
init_lattice = envelope_lattice +1
availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
voxel_coordinates= envelope_lattice.centroids
voxel_coordinates_full_lattice = init_lattice.centroids
flattened_lattice = envelope_lattice.flatten()

In [10]:
Y_coordinates= voxel_coordinates.T[1].flatten()

In [11]:
Full_lattice_Y_coordinates = voxel_coordinates_full_lattice.T[1].flatten()

In [12]:
Building_one =[]
Building_one_indexes =[]
Building_two_three =[]
Building_two_three_indexes =[]
Building_four =[]
Building_four_indexes =[]
not_in_a_building =[]
for center,coordinate,index in zip(Full_lattice_Y_coordinates,voxel_coordinates_full_lattice,range(len(Full_lattice_Y_coordinates))):
    if center >= 18 and center <= 30:
        Building_one.append(coordinate)
        Building_one_indexes.append(index)
        #print("1st")
    elif center >= 40 and center <= 82:
        Building_two_three.append(coordinate)
        Building_two_three_indexes.append(index)
       # print("2nd")
    elif center >= 82:
        Building_four.append(coordinate)
        Building_four_indexes.append(index)
        #print("3rd")
    else:
        not_in_a_building.append(coordinate)
    

# Visualize the Context Mesh + Envelope Lattice

In [13]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes

# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1564.3134508410321, 1810.0972795763837, 1618.43376639279),
 (-142.0987091064453, 103.68511962890625, -87.9783935546875),
 (0.0, 0.0, 1.0)]

# Visibility_vectors

# Compute Intersection of Visibility lines with Context Mesh

### Preparing the List of Ray Directions and Origins

In [14]:
# constructing the sun direction from the sun vectors in a numpy array
visibility_dirs_IJ = np.array(Visibility_sources_for_IJ)
visibility_dirs_Road = np.array(Visibility_sources_for_road)

# For IJ 
ray_dir_building_one_IJ = np.tile(visibility_dirs_IJ, [len(Building_one),1])
ray_src_building_one_IJ = np.tile(Building_one, [1, len(visibility_dirs_IJ)]).reshape(-1, 3)
# For Road
ray_dir_building_one_Road = np.tile(visibility_dirs_Road, [len(Building_one),1])
ray_src_building_one_Road = np.tile(Building_one, [1, len(visibility_dirs_Road)]).reshape(-1, 3)

# For IJ
ray_dir_building_two_and_three_IJ = np.tile(visibility_dirs_IJ, [len(Building_two_three),1])
ray_src_building_two_and_three_IJ = np.tile(Building_two_three, [1, len(visibility_dirs_IJ)]).reshape(-1, 3)
# For Road
ray_dir_building_two_and_three_Road = np.tile(visibility_dirs_Road, [len(Building_two_three),1])
ray_src_building_two_and_three_Road = np.tile(Building_two_three, [1, len(visibility_dirs_Road)]).reshape(-1, 3)

# For IJ 
ray_dir_building_four_IJ = np.tile(visibility_dirs_IJ, [len(Building_four),1])
ray_src_building_four_IJ = np.tile(Building_four, [1, len(visibility_dirs_IJ)]).reshape(-1, 3)
# For Road
ray_dir_building_four_Road = np.tile(visibility_dirs_Road, [len(Building_four),1])
ray_src_building_four_Road = np.tile(Building_four, [1, len(visibility_dirs_Road)]).reshape(-1, 3)



# Computing the Intersection

In [15]:
# computing the intersections of rays with the context mesh for IJ
tri_id_building_one_IJ, ray_id_building_one_IJ = Context_mesh_for_building_one_mesh.ray.intersects_id(ray_origins=ray_src_building_one_IJ, ray_directions=ray_dir_building_one_IJ, multiple_hits=False)
tri_id_building_two_and_three_IJ, ray_id_building_two_and_three_IJ = Context_mesh_for_building_two_and_three_mesh.ray.intersects_id(ray_origins=ray_src_building_two_and_three_IJ, ray_directions=ray_dir_building_two_and_three_IJ, multiple_hits=False)
tri_id_building_four_IJ, ray_id_building_four_IJ = Context_mesh_for_building_four_mesh.ray.intersects_id(ray_origins=ray_src_building_four_IJ, ray_directions=ray_dir_building_four_IJ, multiple_hits=False)


In [16]:
# computing the intersections of rays with the context mesh for IJ
tri_id_building_one_road, ray_id_building_one_Road = Context_mesh_for_building_one_mesh.ray.intersects_id(ray_origins=ray_src_building_one_Road, ray_directions=ray_dir_building_one_Road, multiple_hits=False)
tri_id_building_two_and_three_road, ray_id_building_two_and_three_Road = Context_mesh_for_building_two_and_three_mesh.ray.intersects_id(ray_origins=ray_src_building_two_and_three_Road, ray_directions=ray_dir_building_two_and_three_Road, multiple_hits=False)
tri_id_building_four_road, ray_id_building_four_Road = Context_mesh_for_building_four_mesh.ray.intersects_id(ray_origins=ray_src_building_four_Road, ray_directions=ray_dir_building_four_Road, multiple_hits=False)


## Aggregate Simulation Result in the Visbility Lattice

###  Compute the percentage of time that each voxel sees the Points of interest

In [17]:
def percentage_of_time_each_voxel_sees_the_sun(ray_dir,ray_id,sun_dirs,vox_cens):
    # initializing the hits list full of zeros
    hits = [0]*len(ray_dir)
    # setting the rays that had an intersection to 1
    for id in ray_id:
        hits[id] = 1

    sun_count = len(sun_dirs)
    vox_count = len(vox_cens)
    # initiating the list of ratio
    vox_sun_acc = []
    # iterate over the voxels
    for v_id in range(vox_count):
        # counter for the intersection
        int_count = 0
        # iterate over the sun rays
        for s_id in range(sun_count):
            # computing the ray id from voxel id and sun id
            r_id = v_id * sun_count + s_id

            # summing the intersections
            int_count += hits[r_id]

        # computing the percentage of the rays that DID NOT have 
        # an intersection (aka could see the sun)
        sun_access = 1.0 - int_count/sun_count

        # add the ratio to list
        vox_sun_acc.append(sun_access)

    hits = np.array(hits)
    vox_sun_acc = np.array(vox_sun_acc)

    return hits , vox_sun_acc




In [18]:
percent_for_building_one_IJ= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_one_IJ,ray_id_building_one_IJ,visibility_dirs_IJ,Building_one)
percent_for_building_two_and_three_IJ= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_two_and_three_IJ,ray_id_building_two_and_three_IJ,visibility_dirs_IJ,Building_two_three)
percent_for_building_Four_IJ= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_four_IJ,ray_id_building_four_IJ,visibility_dirs_IJ,Building_four)

In [19]:
percent_for_building_one_Road= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_one_Road,ray_id_building_one_Road,visibility_dirs_Road,Building_one)
percent_for_building_two_and_three_Road= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_two_and_three_Road,ray_id_building_two_and_three_Road,visibility_dirs_Road,Building_two_three)
percent_for_building_Four_Road= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_four_Road,ray_id_building_four_Road,visibility_dirs_Road,Building_four)

#  Store Visibility information in a Lattice IJ

In [20]:
Visbility_IJ_Dummy_lattice = np.copy(Full_lattice_Y_coordinates)

In [21]:
for index,value in zip(Building_one_indexes,percent_for_building_one_IJ[1]):
    Visbility_IJ_Dummy_lattice[index]= value

In [22]:
for index,value in zip(Building_two_three_indexes,percent_for_building_two_and_three_IJ[1]):
    Visbility_IJ_Dummy_lattice[index]= value

In [23]:
for index,value in zip(Building_four_indexes,percent_for_building_Four_IJ[1]):
    Visbility_IJ_Dummy_lattice[index]= value

In [24]:
Visibility_IJ_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Visbility_IJ_Dummy_lattice))])

In [25]:
Visibility_IJ_lattice_padded_np =  Visibility_IJ_lattice_padded.reshape(envelope_lattice.shape)

In [26]:
Visibility_IJ_lattice = tg.to_lattice(Visibility_IJ_lattice_padded_np, Visibility_IJ_lattice_padded_np.shape)

#  Store Visibility information in a Lattice IJ

In [27]:
Visbility_Road_Dummy_lattice = np.copy(Full_lattice_Y_coordinates)

In [28]:
for index,value in zip(Building_one_indexes,percent_for_building_one_Road[1]):
    Visbility_Road_Dummy_lattice[index]= value

In [29]:
for index,value in zip(Building_two_three_indexes,percent_for_building_two_and_three_Road[1]):
    Visbility_Road_Dummy_lattice[index]= value

In [30]:
for index,value in zip(Building_four_indexes,percent_for_building_Four_Road[1]):
    Visbility_Road_Dummy_lattice[index]= value

In [31]:
Visibility_Road_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Visbility_Road_Dummy_lattice))])

In [32]:
Visibility_Road_lattice_padded_np =  Visibility_Road_lattice_padded.reshape(envelope_lattice.shape)

In [33]:
Visibility_Road_lattice = tg.to_lattice(Visibility_Road_lattice_padded_np, Visibility_Road_lattice_padded_np.shape)

# Vizualise the Visibility lattice IJ

In [69]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Visibility to IJ"] = Visibility_IJ_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="Spectral", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.add_points( Visibility_sources_for_IJ, color='#ffa500')
p.show(use_ipyvtk=True,screenshot='Visibility_to_IJ.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

In [67]:
def_cam = p.camera_position

In [68]:
def_cam

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]

# Vizualise the Visibility lattice Road

In [70]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Visibility to Road"] = Visibility_Road_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="YlGnBu", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.add_points( Visibility_sources_for_road, color='#ffa500')
p.show(use_ipyvtk=True,screenshot='Visibility_to_Road.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Save Pickle Lattice

In [36]:
Visibility_to_IJ = pickle.dump( Visibility_IJ_lattice, open( "Visibility_IJ_lattice.p", "wb" ) )

In [37]:
Visibility_to_Road = pickle.dump( Visibility_Road_lattice, open( "Visibility_Road_lattice.p", "wb" ) )